# Process Framework Notebook

## Simple Process

![](simple.png)

### Step 1: Importing the Required Libraries

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.21.1"
#r "nuget: Microsoft.SemanticKernel.Process.Core, 1.21.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.Abstractions, 1.21.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.LocalRuntime, 1.21.1-alpha"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Process;
using System.ComponentModel;
using System.Reflection;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.DependencyInjection;

var builder = Kernel.CreateBuilder();

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0080

ProcessBuilder process = new("BasicChatBot");

## Setup the Classes for the Account Process

### Step 2: Define the Account Process Step Classes


In [7]:
#pragma warning disable SKEXP0001, SKEXP0010,SKEXP0080

public static class CommonEvents
{
    public static readonly string UserInputReceived = nameof(UserInputReceived);
    public static readonly string AssistantResponseGenerated = nameof(AssistantResponseGenerated);
}

public class ScriptedUserInputStep : KernelProcessStep<UserInputState>
{
    public static class Functions
    {
        public const string GetUserInput = nameof(GetUserInput);
    }
    protected UserInputState? _state;
    public virtual void PopulateUserInputs()
    {
        return;
    }
    public override ValueTask ActivateAsync(KernelProcessStepState<UserInputState> state)
    {
        state.State ??= new();
        _state = state.State;

        PopulateUserInputs();

        return ValueTask.CompletedTask;
    }

    /// <summary>
    /// Gets the user input.
    /// </summary>
    [KernelFunction(Functions.GetUserInput)]
    public async ValueTask GetUserInputAsync(KernelProcessStepContext context)
    {
        var userMessage = _state!.UserInputs[_state.CurrentInputIndex];
        _state.CurrentInputIndex++;

        Console.ForegroundColor = ConsoleColor.Yellow;
        Console.WriteLine($"USER: {userMessage}");
        Console.ResetColor();

        // Emit the user input
        await context.EmitEventAsync(new() { Id = CommonEvents.UserInputReceived, Data = userMessage });
    }
}

public record UserInputState
{
    public List<string> UserInputs { get; init; } = [];

    public int CurrentInputIndex { get; set; } = 0;
}

public class DisplayAssistantMessageStep : KernelProcessStep
{
    public static class Functions
    {
        public const string DisplayAssistantMessage = nameof(DisplayAssistantMessage);
    }

    [KernelFunction(Functions.DisplayAssistantMessage)]
    public async ValueTask DisplayAssistantMessageAsync(KernelProcessStepContext context, string assistantMessage)
    {
        Console.ForegroundColor = ConsoleColor.Blue;
        Console.WriteLine($"ASSISTANT: {assistantMessage}\n");
        Console.ResetColor();

        // Emit the assistantMessageGenerated
        await context.EmitEventAsync(new() { Id = CommonEvents.AssistantResponseGenerated, Data = assistantMessage });
    }
}

private sealed class IntroStep : KernelProcessStep
{
    /// <summary>
    /// Prints an introduction message to the console.
    /// </summary>
    [KernelFunction]
    public void PrintIntroMessage()
    {
        System.Console.WriteLine("Welcome to Processes in Semantic Kernel.\n");
    }
}
    //Setup ChatUserInput Step to get the chat input
    private sealed class ChatUserInputStep : ScriptedUserInputStep
    {
        public override void PopulateUserInputs()
        {
            if (_state != null)
            {
                _state.UserInputs.Add("Hello");
                _state.UserInputs.Add("How tall is the tallest mountain?");
                _state.UserInputs.Add("How low is the lowest valley?");
                _state.UserInputs.Add("How wide is the widest river?");
                _state.UserInputs.Add("exit");
            }
        }
    }

    private sealed class ChatBotResponseStep : KernelProcessStep<ChatBotState>
    {
        public static class Functions
        {
            public const string GetChatResponse = nameof(GetChatResponse);
        }

        internal ChatBotState? _state;

        public override ValueTask ActivateAsync(KernelProcessStepState<ChatBotState> state)
        {
            _state = state.State ?? new();
            _state.ChatMessages ??= new();
            return ValueTask.CompletedTask;
        }

        [KernelFunction(Functions.GetChatResponse)]
        public async Task GetChatResponseAsync(KernelProcessStepContext context, string userMessage, Kernel _kernel)
        {
            _state!.ChatMessages.Add(new(AuthorRole.User, userMessage));
            IChatCompletionService chatService = _kernel.Services.GetRequiredService<IChatCompletionService>();
            ChatMessageContent response = await chatService.GetChatMessageContentAsync(_state.ChatMessages).ConfigureAwait(false);
            if (response == null)
            {
                throw new InvalidOperationException("Failed to get a response from the chat completion service.");
            }

            System.Console.ForegroundColor = ConsoleColor.Yellow;
            System.Console.Write("Assistant: ");
            System.Console.ResetColor();
            System.Console.WriteLine(response.Content);

            // Update state with the response
            _state.ChatMessages.Add(response);

            // emit event: assistantResponse
            await context.EmitEventAsync(new KernelProcessEvent { Id = ChatBotEvents.AssistantResponseGenerated, Data = response });
        }
    }
    private sealed class ChatBotState
    {
        internal ChatHistory ChatMessages { get; set; } = new();
    }
    private static class ChatBotEvents
    {
        public const string StartProcess = "startProcess";
        public const string IntroComplete = "introComplete";
        public const string AssistantResponseGenerated = "assistantResponseGenerated";
        public const string Exit = "exit";
    }

### Step 3: Create the Step Objects

In [8]:
#pragma warning disable SKEXP0001, SKEXP0010,SKEXP0080

var introStep = process.AddStepFromType<IntroStep>();
var userInputStep = process.AddStepFromType<ChatUserInputStep>();
var responseStep = process.AddStepFromType<ChatBotResponseStep>();

### Step 4: Define the patterns for each step

In [5]:
#pragma warning disable SKEXP0001, SKEXP0010,SKEXP0080

process
    .OnExternalEvent(ChatBotEvents.StartProcess)
    .SendEventTo(new ProcessFunctionTargetBuilder(introStep));

// When the intro is complete, notify the userInput step
introStep
    .OnFunctionResult(nameof(IntroStep.PrintIntroMessage))
    .SendEventTo(new ProcessFunctionTargetBuilder(userInputStep));

// When the userInput step emits an exit event, send it to the end steprt
userInputStep
    .OnFunctionResult("GetUserInput")
    .StopProcess();

// When the userInput step emits a user input event, send it to the assistantResponse step
userInputStep
    .OnEvent(CommonEvents.UserInputReceived)
    .SendEventTo(new ProcessFunctionTargetBuilder(responseStep, parameterName: "userMessage"));

// When the assistantResponse step emits a response, send it to the userInput step
responseStep
    .OnEvent(ChatBotEvents.AssistantResponseGenerated)
    .SendEventTo(new ProcessFunctionTargetBuilder(userInputStep));

KernelProcess kernelProcess = process.Build();

### Start the Process

In [ ]:
 #pragma warning disable SKEXP0001, SKEXP0010,SKEXP0080
 
// Start the process with an initial external event
var runningProcess = await kernelProcess.StartAsync(kernel, new KernelProcessEvent() { Id = ChatBotEvents.StartProcess, Data = null });